### Model 1.5: account for different traffic and pricing depending on time of day

In [53]:
from amplpy import AMPL
import pandas as pd

In [54]:
rideshares = pd.read_csv('./rideshare_tg22.csv')
airports = ['LAX', 'ONT', 'SNA', 'BUR']
car_types = ['UberX', 'UberXL', 'Uber Green']

In [55]:
ez_rideshares = rideshares#[rideshares['Departure Date'] == '5/11/24']
ez_rideshares.rename(columns={'Departure Time (Pacific Daylight Time)': 'Flight depart'}, inplace=True)

In [56]:
cost_data = {
    ('LAX', 'UberX'): 90, ('LAX', 'UberXL'): 120, ('LAX', 'Uber Green'): 91,
    ('ONT', 'UberX'): 30, ('ONT', 'UberXL'): 40, ('ONT', 'Uber Green'): 31,
    ('SNA', 'UberX'): 90, ('SNA', 'UberXL'): 120, ('SNA', 'Uber Green'): 91,
    ('BUR', 'UberX'): 90, ('BUR', 'UberXL'): 120, ('BUR', 'Uber Green'): 91
}

In [57]:
capacity_data = {
    'UberX': 4,
    'UberXL': 6,
    'Uber Green': 4
}

In [58]:
emissions_data = {
    'UberX': .347*1.69,
    'UberXL': 1.2*.347*1.69,
    'Uber Green': .45*.347*1.69
}
# kg CO2 per passenger mile
# UberX: https://www.uber.com/us/en/about/reports/sustainability-report/
# UberXL multiplier: https://www.greencarreports.com/news/1143316_versus-cars-suvs-emit-about-20-more-co2-ev-or-not
# Uber Green multiplier: https://www.uber.com/us/en/ride/ubergreen/
# 1.69 multiplier: https://fortune.com/2024/03/12/uber-carbon-dioxide-emissions-co2-savings-electric-hybrid-cars/

In [59]:
travel_time_data = {
    'LAX': 1.5,
    'ONT': 0.5,
    'SNA': 1.5,
    'BUR': 1.5
}

In [60]:
trip_length_data = {
    'LAX': 50.4,
    'ONT': 9.1,
    'SNA': 38.3,
    'BUR': 41.0
}

In [61]:
ez_rideshares['Flight depart'] = pd.to_datetime(ez_rideshares['Flight depart'], format='%H:%M')
ez_rideshares['Flight times'] = ez_rideshares['Flight depart'].dt.hour + 0.5 * (ez_rideshares['Flight depart'].dt.minute//30)

In [70]:
ampl = AMPL()
ampl.eval(r"""
    set A;
    set S;
    set P = 0.25..24 by 0.25;  # time of arrival to airport
    set C;

    param cost{A, C};
    param capacity{C};
    param travelTime{A};
    param flightTime{S, A};
    param emissions{C};
    param milesTo{A};

    var numCars{A, P, C} >= 0 integer;  # x
    var leaving{A, S, P} binary;  # 1 if student s leaves for airport a at time p, else 0
    var departureTime{A, S};
    var totalEmissions;
    var tripCost;

    minimize objective: tripCost + 0.185*totalEmissions; # co2 cost: https://www.nature.com/articles/s41586-022-05224-9
          
    subject to calculateCost:
        tripCost = sum{a in A, p in P, c in C} numCars[a, p, c]*cost[a, c];
          
    subject to calculateEmissions:
        totalEmissions = sum{a in A, p in P, c in C} numCars[a, p, c] * emissions[c] * milesTo[a];

    subject to notTooLateConstraint{a in A, s in S, p in P}:
        p - leaving[a, s, p]*(flightTime[s, a] - 2 - travelTime[a]) <= 100*(1 - leaving[a, s, p]);

    subject to notTooEarlyConstraint{a in A, s in S, p in P}:
        leaving[a, s, p]*(flightTime[s, a] - 4 - travelTime[a]) - p <= 100*(1 - leaving[a, s, p]);

    subject to capacityConstraint{a in A, p in P}:
        sum{s in S} leaving[a, s, p] <= sum{c in C} numCars[a, p, c] * capacity[c];

    subject to onlyOneActiveY{s in S}:
        sum{a in A, p in P} leaving[a, s, p] = 1;

    subject to defineDepartureTime{a in A, s in S}:
        departureTime[a, s] = sum{p in P} leaving[a, s, p] * p;

""")

In [63]:
# reset index on ez_rideshares to start at 0
ez_rideshares.reset_index(drop=True, inplace=True)

In [64]:
# set flight time data equal to Flight Times form ez_rideshares
# format: {(ez_rideshares['Name'][i], 'ONT') : ez_rideshares['Flight times'][i]}

raw_flight_time_data = {}
for i in range(len(ez_rideshares)):
    raw_flight_time_data[(ez_rideshares['Name'][i], ez_rideshares['Airport'][i])] = ez_rideshares['Flight times'][i]

In [65]:
flight_time_data = {}
for s in ez_rideshares['Name']:
    for j in airports:
        if (s, j) in raw_flight_time_data:
            flight_time_data[(s, j)] = raw_flight_time_data[(s, j)]
        else:
            flight_time_data[(s, j)] = 0

In [72]:
# read ez_rideshares into ampl2
ampl.set['A'] = airports
ampl.set['S'] = ez_rideshares['Name']
ampl.set['C'] = car_types

ampl.getParameter("cost").setValues(cost_data)
ampl.getParameter("capacity").setValues(capacity_data)
ampl.getParameter("travelTime").setValues(travel_time_data)
ampl.getParameter("flightTime").setValues(flight_time_data)
ampl.getParameter("emissions").setValues(emissions_data)
ampl.getParameter('milesTo').setValues(trip_length_data)


In [73]:
# run the model
ampl.solve(solver='gurobi')

Gurobi 11.0.3:Gurobi 11.0.3: optimal solution; objective 569.3724268
190 simplex iterations
1 branching node
absmipgap=1.13687e-13, relmipgap=0


In [74]:
# display the results
student_airport_times = {}
for student in ez_rideshares['Name']:
    for a in airports:
        x = ampl.getValue(f'departureTime["{a}", "{student}"]')
        if x != 0:
            student_airport_times[student] = (a, x)
student_airport_times

{'Nithya Yeluri': ('ONT', 2.5),
 'Nina Jobanputra': ('ONT', 9.5),
 'Aanya Pratapneni ': ('ONT', 9.5),
 'Will Sedo': ('ONT', 9.5),
 'Marcella Todd': ('ONT', 11.5),
 'Kaanthi Pandhigunta': ('ONT', 11.5),
 'Cevi Bainton': ('ONT', 11.5),
 'Ally Dye': ('ONT', 11.5),
 'Maya Maranto': ('ONT', 15),
 'Alec Vercruysse': ('ONT', 15),
 'Rebecca Preis': ('ONT', 15),
 'Susan Li': ('ONT', 15),
 'Tanvi Krishnan ': ('ONT', 16.5),
 'Jennifer Li': ('ONT', 16.5),
 'Allison Marten': ('ONT', 16.5),
 'Aldrin Feliciano': ('ONT', 16.5),
 'Julianna Schalkwyk': ('ONT', 19),
 'Charles Weismann': ('ONT', 2.5),
 'Katrina Nguyen': ('ONT', 2.5),
 'Bennet Matazzoni': ('ONT', 11.5),
 'Thaxter Shaw': ('LAX', 17),
 'Alicia Krasner': ('LAX', 7.5),
 'Laura Vairus': ('LAX', 12),
 'Ashley Kim': ('SNA', 15.5)}

In [75]:
for i in range(1, 49):
    for a in airports:
        for car in car_types:
            numCars = ampl.getValue(f"numCars['{a}', {i/2}, '{car}']")
            if numCars > 0:
                print(f'Time {i/2} has {numCars} {car} car(s) to {a}')

Time 2.5 has 1 UberX car(s) to ONT
Time 7.5 has 1 Uber Green car(s) to LAX
Time 9.5 has 1 UberX car(s) to ONT
Time 11.5 has 1 UberXL car(s) to ONT
Time 12.0 has 1 Uber Green car(s) to LAX
Time 15.0 has 1 UberX car(s) to ONT
Time 15.5 has 1 Uber Green car(s) to SNA
Time 16.5 has 1 UberX car(s) to ONT
Time 17.0 has 1 Uber Green car(s) to LAX
Time 19.0 has 1 UberX car(s) to ONT
